In [4]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
!mkdir -p ~/.kaggle
!cp "/content/drive/Shareddrives/BTT - Mandelic Acid/kaggle.json" ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c bttai-ajl-2025 -p /content/
!kaggle datasets download -d <dataset-name> -p /content/
!unzip -q /content/bttai-ajl-2025.zip -d /content/bttai-ajl-2025

 97% 298M/306M [00:03<00:00, 81.2MB/s]
100% 306M/306M [00:03<00:00, 89.6MB/s]
/bin/bash: line 1: dataset-name: No such file or directory


In [8]:
# 1. Import Necessary Libraries
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Explanation:
# - pandas and numpy: for data manipulation
# - sklearn: for splitting data and encoding labels
# - tensorflow.keras: for building and training the neural network

In [9]:
# 2. Load Data
train_df = pd.read_csv('/content/bttai-ajl-2025/train.csv')
test_df = pd.read_csv('/content/bttai-ajl-2025/test.csv')

# Add .jpg extension to md5hash column to reference the file_name
train_df['md5hash'] = train_df['md5hash'].astype(str) + '.jpg'
test_df['md5hash'] = test_df['md5hash'].astype(str) + '.jpg'

# Combine label and md5hash to form the correct path
train_df['file_path'] = train_df['label'] + '/' + train_df['md5hash']

In [10]:
# Check the first few rows to understand the structure
print(train_df.head())

                                md5hash  fitzpatrick_scale  \
0  fd06d13de341cc75ad679916c5d7e6a6.jpg                  4   
1  a4bb4e5206c4e89a303f470576fc5253.jpg                  1   
2  c94ce27e389f96bda998e7c3fa5c4a2e.jpg                  5   
3  ebcf2b50dd943c700d4e2b586fcd4425.jpg                  3   
4  c77d6c895f05fea73a8f3704307036c0.jpg                  1   

   fitzpatrick_centaur                             label nine_partition_label  \
0                    4                 prurigo-nodularis     benign-epidermal   
1                    1  basal-cell-carcinoma-morpheiform  malignant-epidermal   
2                    5                            keloid         inflammatory   
3                    3              basal-cell-carcinoma  malignant-epidermal   
4                    1                 prurigo-nodularis     benign-epidermal   

  three_partition_label            qc  ddi_scale  \
0                benign           NaN         34   
1             malignant           Na

In [11]:
# 3. Data Preprocessing
# Encode the labels
label_encoder = LabelEncoder()
train_df['encoded_label'] = label_encoder.fit_transform(train_df['label'])

# Split the data into training and validation sets
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=42)

# Define image data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Define the directory paths
train_dir = '/content/bttai-ajl-2025/train/train/'

In [12]:
def create_generator(dataframe, directory, batch_size=32, target_size=(128, 128)):
    """
    Template function to create image generators.
    Students should complete this function to load images and labels properly.
    """
    # Fill in the correct flow_from_dataframe parameters
    generator = train_datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col='file_path',  # Use combined path
        y_col='encoded_label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='raw',
        validate_filenames=False  # Disable strict filename validation
    )
    return generator

In [13]:
# Create generators
train_generator = create_generator(train_data, train_dir)
val_generator = create_generator(val_data, train_dir)

Found 2288 non-validated image filenames.
Found 572 non-validated image filenames.


In [14]:
# TODO: You should implement the model architecture here.
# Feel free to explore different model types that best serve your purpose.

model = Sequential([
    # First Convolutional Layer
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2,2)),

    # Second Convolutional Layer
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    # Third Convolutional Layer
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),

    # Flatten Layer
    Flatten(),

    # Fully Connected Layers
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),

    Dense(len(train_df['label'].unique()), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 21)                  │           1,365 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,314,261 (12.64 MB)

 Trainable params: 3,314,261 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
train_df.columns

Index(['md5hash', 'fitzpatrick_scale', 'fitzpatrick_centaur', 'label',
       'nine_partition_label', 'three_partition_label', 'qc', 'ddi_scale',
       'file_path', 'encoded_label'],
      dtype='object')

In [16]:
test_df.columns

Index(['md5hash', 'fitzpatrick_scale', 'fitzpatrick_centaur', 'qc',
       'ddi_scale'],
      dtype='object')

In [17]:
# TODO: Train your model here.
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 101s 1s/step - accuracy: 0.0631 - loss: 3.0311 - val_accuracy: 0.1678 - val_loss: 2.8630
Epoch 2/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.1109 - loss: 2.9251 - val_accuracy: 0.1678 - val_loss: 2.7978
Epoch 3/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 95s 1s/step - accuracy: 0.1275 - loss: 2.8638 - val_accuracy: 0.1748 - val_loss: 2.7655
Epoch 4/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.1529 - loss: 2.8347 - val_accuracy: 0.1783 - val_loss: 2.7537
Epoch 5/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 92s 1s/step - accuracy: 0.1383 - loss: 2.7989 - val_accuracy: 0.1958 - val_loss: 2.6955
Epoch 6/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 96s 1s/step - accuracy: 0.1580 - loss: 2.7679 - val_accuracy: 0.2150 - val_loss: 2.6870
Epoch 7/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 138s 1s/step - accuracy: 0.1760 - loss: 2.7176 - val_accuracy: 0.2115 - val_loss: 2.6347
Epoch 8/10
72/72 ━━━━━━━━━━━━━━━━━━━━ 93s 1s/step - accuracy: 0.1934 - loss: 2.6323 - val_accuracy: 0.2273 - val_lo

In [1]:
# 6. Make Predictions on Test Data
def preprocess_test_data(test_df, directory):
    """
    Template for loading and preprocessing test images.
    """
    # TODO: create a generator for the test set here.
    # test_datagen = ImageDataGenerator(rescale=1./255)
    # test_generator = test_datagen.flow_from_dataframe(
    #     .... ## set the test_generator here
    # )F
    # return test_generator

In [18]:
# Load test data
test_dir = '/content/bttai-ajl-2025/test/test/'
# test_generator = preprocess_test_data(test_df, test_dir)

In [19]:
# Generate predictions based on the trained model
# Then, save the predictions into a CSV file for submission

# Complete Prediction and Submission Pipeline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# 1. Create test generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='md5hash',
    y_col=None,
    target_size=(128, 128),
    batch_size=32,
    class_mode=None,
    shuffle=False,
    validate_filenames=False
)

# 2. Generate predictions
predictions = model.predict(test_generator, verbose=1)
predicted_classes = np.argmax(predictions, axis=1)

# 3. Decode labels back to original names
predicted_labels = label_encoder.inverse_transform(predicted_classes)

# 4. Save predictions to CSV for submission
submission_df = pd.DataFrame({
    'md5hash': test_df['md5hash'].str.replace('.jpg', '', regex=False),
    'label': predicted_labels
})

# Save to CSV
submission_df.to_csv('submission.csv', index=False)

print(submission_df.head())



Found 1227 non-validated image filenames.
39/39 ━━━━━━━━━━━━━━━━━━━━ 18s 452ms/step
                            md5hash                    label
0  0844ae634f0e6e7ef1f73c2aeecbae0e            acne-vulgaris
1  3b290d262098f761d719aa07cf36c040     basal-cell-carcinoma
2  cf561d08ac46d0fda678bff6621005ee     basal-cell-carcinoma
3  e6371069be05c6b0a95b4b3f1bacc9a5            acne-vulgaris
4  f76cddb37265f97508f159078dcc7e7c  squamous-cell-carcinoma
